In [628]:
import os
import pulp
import pandas as pd
import time
import numpy as np
import sys
from datetime import datetime, timedelta
import pyodbc
import datetime

In [629]:
now = datetime.datetime.now()

In [630]:
starting_time = now.strftime("%Y-%m-%d %H:%M:%S")
starting_time = datetime.datetime.fromisoformat(starting_time)

In [631]:
print(f"{starting_time}")

2024-06-21 14:31:44


In [632]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=STEELDNA;'
                      'Database=DynaMix;'
                      'UID=sa;'
                      'PWD=admin@123;'
                      'Trusted_Connection=no;')

In [633]:
cursor = conn.cursor()

In [634]:
submitted_user_inputs = pd.read_sql('SELECT * FROM DynaMix.dbo.submitted_user_inputs', conn)


C:\Users\1090384\AppData\Local\Temp\ipykernel_1388\1937519029.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  submitted_user_inputs = pd.read_sql('SELECT * FROM DynaMix.dbo.submitted_user_inputs', conn)


In [635]:
previous_run = pd.read_sql('SELECT TOP(1000) HEAT_NUMBER, MAX(HM_WGT_KG) AS HM_WGT_KG, MIN(START_TIME) AS START_TIME FROM DynaMix.dbo.SIF_FB_EAF_DASHBOARD GROUP BY HEAT_NUMBER ORDER BY START_TIME desc', conn)

C:\Users\1090384\AppData\Local\Temp\ipykernel_1388\39156273.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  previous_run = pd.read_sql('SELECT TOP(1000) HEAT_NUMBER, MAX(HM_WGT_KG) AS HM_WGT_KG, MIN(START_TIME) AS START_TIME FROM DynaMix.dbo.SIF_FB_EAF_DASHBOARD GROUP BY HEAT_NUMBER ORDER BY START_TIME desc', conn)


In [636]:
previous_run.head(2)

,HEAT_NUMBER,HM_WGT_KG,START_TIME
0,24202571,124248.0,2024-06-21 14:29:26
1,24302203,0.0,2024-06-21 14:14:57


In [637]:
pi_server_df = pd.read_sql('SELECT * FROM DynaMix.dbo.Pi_server', conn)

C:\Users\1090384\AppData\Local\Temp\ipykernel_1388\4199547632.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pi_server_df = pd.read_sql('SELECT * FROM DynaMix.dbo.Pi_server', conn)


In [638]:
pi_server_df.head(5)

,MSG_FLAG,TAG,TIME,VALUE,PLANT,UNIT,TAG_DESCRIPTION
0,N,JSW_DLV_BF1_CSTHS_TH4_DPSIDE,2024-06-21 11:14:40,9.0,BF1,NO,LADDLE NO TP4
1,N,JSW_DLV_BF1_CSTHS_TH4_CRSIDE,2024-06-21 11:14:40,33.0,BF1,NO,LADDLE NO TP4
2,N,JSW_DLV_BF1_CSTHS_TH3_DPSIDE,2024-06-21 11:14:40,9.0,BF1,NO,LADDLE NO TP3
3,N,JSW_DLV_BF1_CSTHS_TH3_CRSIDE,2024-06-21 11:14:40,31.0,BF1,NO,LADDLE NO TP3
4,N,JSW_DLV_BF1_CSTHS_TH2_DPSIDE,2024-06-21 11:14:40,3.0,BF1,NO,LADDLE NO TP2


In [639]:
production_rate_latest = pi_server_df[pi_server_df['TAG_DESCRIPTION']=="BF Production Rate"].sort_values(by="TIME", ascending=False).drop_duplicates(subset=['TAG'],keep="first")

In [640]:
bf_rate = float(production_rate_latest[production_rate_latest['TAG_DESCRIPTION']=="BF Production Rate"]['VALUE'])

C:\Users\1090384\AppData\Local\Temp\ipykernel_1388\155204185.py:1: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  bf_rate = float(production_rate_latest[production_rate_latest['TAG_DESCRIPTION']=="BF Production Rate"]['VALUE'])


In [641]:
sif_frl3_torpedo_df = pd.read_sql('SELECT * FROM DynaMix.dbo.SIF_FrL3_TOPRPEDO order by INSERT_DATE DESC', conn)

C:\Users\1090384\AppData\Local\Temp\ipykernel_1388\1888726077.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sif_frl3_torpedo_df = pd.read_sql('SELECT * FROM DynaMix.dbo.SIF_FrL3_TOPRPEDO order by INSERT_DATE DESC', conn)


In [642]:
sif_frl3_torpedo_df['INSERT_DATE'] = pd.to_datetime(sif_frl3_torpedo_df['INSERT_DATE'])
sif_frl3_torpedo_df['DATE_IN'] = pd.to_datetime(sif_frl3_torpedo_df['DATE_IN'])
sif_frl3_torpedo_df['TIME_IN'] = pd.to_datetime(sif_frl3_torpedo_df['TIME_IN'], format='%H:%M:%S').dt.time
sif_frl3_torpedo_df['DATE_OUT'] = pd.to_datetime(sif_frl3_torpedo_df['DATE_OUT'])
sif_frl3_torpedo_df['TIME_OUT'] = pd.to_datetime(sif_frl3_torpedo_df['TIME_OUT'], format='%H:%M:%S').dt.time
sif_frl3_torpedo_df['SEC_TARE_DATETIME'] = pd.to_datetime(sif_frl3_torpedo_df['SEC_TARE_DATETIME'].str[:19],format="%Y-%m-%dT%H:%M:%S")
sif_frl3_torpedo_df['UPDATE_DATE'] = pd.to_datetime(sif_frl3_torpedo_df['UPDATE_DATE'])

In [643]:
sif_frl3_torpedo_df['SEC_TARE_DATETIME']

0                       NaT
1       2024-06-21 13:39:59
2       2024-06-21 13:28:53
3       2024-06-21 12:35:14
4       2024-06-21 12:22:40
                ...        
50192   2021-06-07 12:45:45
50193   2021-06-07 08:07:27
50194   2021-06-07 09:16:03
50195   2021-06-07 20:25:02
50196   2021-06-07 07:20:38
Name: SEC_TARE_DATETIME, Length: 50197, dtype: datetime64[ns]

In [644]:
sif_frl3_torpedo_df.columns

Index(['SERIAL_NO', 'INSERT_DATE', 'MSG_FLAG', 'TRIP_NO', 'TL_NO', 'CAST_NO',
       'DATE_IN', 'TIME_IN', 'DATE_OUT', 'TIME_OUT', 'TARE_WT', 'GROSS_WT',
       'NET_WT', 'SEC_TAREWT', 'SEC_TARE_DATETIME', 'WB_NO', 'SUPPLY_AREA',
       'TAP_NO', 'MSG_STATUS', 'UPDATE_DATE', 'TORPEDO_FLAG', 'SHIFT',
       'SMS_ENTRY', 'SMS_EXIT', 'BF_Entry', 'BF_Exit'],
      dtype='object')

In [645]:
geofence_data = pd.read_sql("SELECT TOP(1000) * FROM Geofence_data WHERE geofencenumber in ('521','515','529') ORDER BY INSERT_DATE desc", conn)

C:\Users\1090384\AppData\Local\Temp\ipykernel_1388\2911712461.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  geofence_data = pd.read_sql("SELECT TOP(1000) * FROM Geofence_data WHERE geofencenumber in ('521','515','529') ORDER BY INSERT_DATE desc", conn)


In [646]:
geofence_data.columns

Index(['geofencenumber', 'lno', 'timeunix', 'latitude', 'longitude',
       'entry_status', 'location', 'SEQ_VALUE', 'INSERT_DATE', 'MSG_FLAG'],
      dtype='object')

In [647]:
geofence_data = geofence_data[geofence_data['lno'].str.contains('TLC')]
geofence_data['TLC_No_Check'] = geofence_data['lno'].str.slice(start=4).astype(int)
geofence_data=geofence_data[(geofence_data['TLC_No_Check']<=15) | (geofence_data['TLC_No_Check']>=30)]

In [648]:
torpedo_geofence_data = geofence_data[geofence_data['INSERT_DATE'] >= starting_time - datetime.timedelta(minutes = 60)]
torpedo_geofence_data.head()

,geofencenumber,lno,timeunix,latitude,longitude,entry_status,location,SEQ_VALUE,INSERT_DATE,MSG_FLAG,TLC_No_Check
0,529,TLC 07,1718123220,18.6897183,73.0409917,101,At Weigh Bridge*,7647178.0,2024-06-21 14:30:06.337,N,7
1,529,TLC 07,1718123160,18.690725,73.0411383,102,At Gate 3 crossing (sms side)*,7647177.0,2024-06-21 14:30:06.327,N,7
8,521,TLC 34,1718122417,18.6846033,73.0373367,103,At SGP Conveyor*,7647115.0,2024-06-21 14:30:05.257,N,34
10,529,TLC 07,1718122320,18.6899617,73.0410067,101,At Weigh Bridge*,7647102.0,2024-06-21 14:30:05.093,N,7
11,515,TLC 07,1718122320,18.6899617,73.0410067,103,At Weigh Bridge*,7647100.0,2024-06-21 14:30:05.090,N,7


In [649]:
geofence_data = geofence_data[geofence_data['INSERT_DATE'] >= starting_time - datetime.timedelta(minutes = 120)]

In [650]:
latest_status = torpedo_geofence_data.groupby('lno')['INSERT_DATE'].transform(max) == torpedo_geofence_data['INSERT_DATE']
#getting latest torpedo timing

In [651]:
bf_geofence = torpedo_geofence_data[latest_status]
bf_geofence
#521 (BF), 529 (WB) or 515 (SMS)

,geofencenumber,lno,timeunix,latitude,longitude,entry_status,location,SEQ_VALUE,INSERT_DATE,MSG_FLAG,TLC_No_Check
0,529,TLC 07,1718123220,18.6897183,73.0409917,101,At Weigh Bridge*,7647178.0,2024-06-21 14:30:06.337,N,7
8,521,TLC 34,1718122417,18.6846033,73.0373367,103,At SGP Conveyor*,7647115.0,2024-06-21 14:30:05.257,N,34
30,515,TLC 14,1718121614,18.6912217,73.04117,101,At Gate 3 crossing (sms side)*,7647026.0,2024-06-21 14:25:05.203,N,14
44,521,TLC 01,1718121004,18.6854317,73.0384767,101,At Tap Hole 4*,7646985.0,2024-06-21 14:25:04.927,N,1
88,521,TLC 02,1718117963,18.6862033,73.039885,101,At Junction Point:18*,7646710.0,2024-06-21 14:10:05.007,N,2
106,521,TLC 11,1718117043,18.6849,73.03855,103,At BF GCP Area*,7646623.0,2024-06-21 14:05:05.120,N,11
121,529,TLC 13,1718116512,18.6898267,73.040885,101,At Weigh Bridge*,7646566.0,2024-06-21 14:00:06.023,N,13
123,529,TLC 31,1718116363,18.690065,73.0409583,103,At Weigh Bridge*,7646551.0,2024-06-21 14:00:05.887,N,31
153,521,TLC 05,1718114025,18.6848817,73.0386967,102,At BF GCP Area*,7646397.0,2024-06-21 13:55:04.950,N,5
154,521,TLC 05,1718114035,18.6850583,73.038575,101,At BF GCP Area*,7646398.0,2024-06-21 13:55:04.950,N,5


In [652]:
bf_geofence = bf_geofence[(bf_geofence['geofencenumber'] == '521') & (bf_geofence['entry_status'] == '103')]
## 102 -> torpedo has entered geofence
# 101 -> torpedo is currently at geofence
# 103 -> torpedo has exited geofence
bf_geofence

,geofencenumber,lno,timeunix,latitude,longitude,entry_status,location,SEQ_VALUE,INSERT_DATE,MSG_FLAG,TLC_No_Check
8,521,TLC 34,1718122417,18.6846033,73.0373367,103,At SGP Conveyor*,7647115.0,2024-06-21 14:30:05.257,N,34
106,521,TLC 11,1718117043,18.6849,73.03855,103,At BF GCP Area*,7646623.0,2024-06-21 14:05:05.120,N,11


In [653]:
# # creating the columns that we want
# this is just some data processing so that we can store all this information in a clean table
bf_geofence['Torpedo'] = bf_geofence['lno'].apply(lambda x: 'Torpedo ' + str(float(x.split(' ')[1])))
bf_geofence.rename(columns = {'INSERT_DATE':'BF Exit Time'}, inplace = True)
bf_geofence = bf_geofence[['Torpedo','BF Exit Time']]
bf_geofence['Weight Bridge Arrival Time'] = ''
bf_geofence['Hot Metal'] = ''
bf_geofence['Arrival Time'] = ''
bf_geofence

C:\Users\1090384\AppData\Local\Temp\ipykernel_1388\542422848.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bf_geofence['Torpedo'] = bf_geofence['lno'].apply(lambda x: 'Torpedo ' + str(float(x.split(' ')[1])))
C:\Users\1090384\AppData\Local\Temp\ipykernel_1388\542422848.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bf_geofence.rename(columns = {'INSERT_DATE':'BF Exit Time'}, inplace = True)
C:\Users\1090384\AppData\Local\Temp\ipykernel_1388\542422848.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

,Torpedo,BF Exit Time,Weight Bridge Arrival Time,Hot Metal,Arrival Time
8,Torpedo 34.0,2024-06-21 14:30:05.257,,,
106,Torpedo 11.0,2024-06-21 14:05:05.120,,,


In [654]:
sms_geofence = torpedo_geofence_data[latest_status]
sms_geofence = sms_geofence[(sms_geofence['geofencenumber'] == '515') & (sms_geofence['entry_status'].isin(['102','101','103']))].copy()
sms_geofence

,geofencenumber,lno,timeunix,latitude,longitude,entry_status,location,SEQ_VALUE,INSERT_DATE,MSG_FLAG,TLC_No_Check
30,515,TLC 14,1718121614,18.6912217,73.04117,101,At Gate 3 crossing (sms side)*,7647026.0,2024-06-21 14:25:05.203,N,14
208,515,TLC 03,1718110376,18.6912183,73.0410567,103,At Gate 3 crossing (sms side)*,7646202.0,2024-06-21 13:45:05.040,N,3


In [655]:
sms_geofence['Torpedo'] = sms_geofence['lno'].apply(lambda x: 'Torpedo ' + str(float(x.split(' ')[1])))
sms_geofence.rename(columns = {'INSERT_DATE':'SMS Entry Time'}, inplace = True)
sms_geofence = sms_geofence[['Torpedo','SMS Entry Time','entry_status']]
sms_geofence

,Torpedo,SMS Entry Time,entry_status
30,Torpedo 14.0,2024-06-21 14:25:05.203,101
208,Torpedo 3.0,2024-06-21 13:45:05.040,103


In [656]:
latest_urn = submitted_user_inputs['URN'][0]
latest_urn

'URN81234'

In [657]:
HM_left_in_SMS_tons = float(submitted_user_inputs['HM_left_in_SMS_tons'][0])
print(HM_left_in_SMS_tons)
previous_run.head()

150.0


,HEAT_NUMBER,HM_WGT_KG,START_TIME
0,24202571,124248.0,2024-06-21 14:29:26
1,24302203,0.0,2024-06-21 14:14:57
2,24402355,134896.0,2024-06-21 13:55:38
3,24102467,142708.0,2024-06-21 13:29:08
4,24302202,174537.0,2024-06-21 13:09:04


In [658]:
previous_start = submitted_user_inputs['Starting_Point'][0]
submitted_user_inputs.head()

,URN,Starting_Point,HM_left_in_SMS_tons,Scrap_to_be_consumed_tons,DRI_Rate,HM_Yield,DRI_Yield,Scrap_Yield,BF_Production_Rate,Shell1_Scrap,...,Shell1_Downtime_Flag,Shell2_Downtime_Flag,Shell3_Downtime_Flag,Shell4_Downtime_Flag,Shell1_Downtime,Shell2_Downtime,Shell3_Downtime,Shell4_Downtime,PowerShell12,PowerShell34
0,URN81234,2024-06-21 14:25:31,150.0,160.0,265.0,87.0,78.0,78.0,406.0,N,...,3,3,3,3,0.0,0.0,0.0,0.0,High Power,High Power


In [659]:
previous_run = previous_run[(previous_run['START_TIME'] >= previous_start) & (previous_run['START_TIME'] <= starting_time)]

In [660]:
previous_run['HM_WGT_TONS'] = previous_run['HM_WGT_KG']/1000

In [661]:
hm_consumed = previous_run['HM_WGT_TONS'].sum()
hm_consumed

124.248

In [662]:
def merging_torpedo_data(row):
    """

    This function will merge the Date out and time out columns in the torpedo table (sif_frl3_torpedo_df)

    Simply concatenating the DATE_IN and TIME_IN to get the weighbridge entry time in datetime format
    Same for DATE_OUT, TIME_OUT as well

    For ex:
    DATE_OUT -> 01/01/21
    TIME_OUT -> 02:00:00

    returns WB_OUT_DATETIME -> 01/01/21 02:00:00


    """
    row['WB_IN_DATETIME'] = row['DATE_IN'].combine(row['DATE_IN'], row['TIME_IN'])
    row['WB_OUT_DATETIME'] = row['DATE_OUT'].combine(row['DATE_OUT'], row['TIME_OUT'])

    return row

In [663]:
# Filtering the dataset for weigh bridge data only
# HSM supply area filter keeps only those torpedoes going to SMS
wb_data = sif_frl3_torpedo_df[(sif_frl3_torpedo_df['SUPPLY_AREA']=="HSM") | (sif_frl3_torpedo_df['SUPPLY_AREA']=="BF2/HSM") | (sif_frl3_torpedo_df['SUPPLY_AREA']=="BF 2/HSM")]
wb_data = wb_data[(wb_data['TL_NO'] <=15) | (wb_data['TL_NO'] >=30) ]

# calling function to combine time_out & date_out
wb_data = wb_data.apply(merging_torpedo_data, axis = 1)

In [664]:
wb_data.sort_values(by="INSERT_DATE",ascending=True).head()

,SERIAL_NO,INSERT_DATE,MSG_FLAG,TRIP_NO,TL_NO,CAST_NO,DATE_IN,TIME_IN,DATE_OUT,TIME_OUT,...,MSG_STATUS,UPDATE_DATE,TORPEDO_FLAG,SHIFT,SMS_ENTRY,SMS_EXIT,BF_Entry,BF_Exit,WB_IN_DATETIME,WB_OUT_DATETIME
50035,91806.0,2021-06-07 18:24:30.853,N,3.0,2.0,10023668,2021-06-07,05:29:26,2021-06-07,06:48:37,...,3.0,NaT,None,None,NaT,NaT,NaT,NaT,2021-06-07 05:29:26,2021-06-07 06:48:37
50034,91807.0,2021-06-07 18:24:30.907,N,2.0,30.0,10023668,2021-06-07,05:50:41,2021-06-07,07:28:30,...,5.0,NaT,None,None,NaT,NaT,NaT,NaT,2021-06-07 05:50:41,2021-06-07 07:28:30
50033,91808.0,2021-06-07 18:24:30.927,N,1.0,9.0,10023670,2021-06-07,06:29:25,2021-06-07,10:13:21,...,3.0,NaT,None,None,NaT,NaT,NaT,NaT,2021-06-07 06:29:25,2021-06-07 10:13:21
50032,91809.0,2021-06-07 18:24:30.950,N,2.0,8.0,10023672,2021-06-07,06:38:05,2021-06-07,14:51:21,...,5.0,NaT,None,None,NaT,NaT,NaT,NaT,2021-06-07 06:38:05,2021-06-07 14:51:21
50031,91810.0,2021-06-07 18:24:30.973,N,3.0,14.0,30023669,2021-06-07,07:20:39,2021-06-07,08:24:52,...,3.0,NaT,None,None,NaT,NaT,NaT,NaT,2021-06-07 07:20:39,2021-06-07 08:24:52


In [665]:
df_1 = wb_data[(wb_data['SMS_ENTRY'] >= previous_start) & (wb_data['SMS_ENTRY'] <= starting_time)]

In [666]:
df_2 = wb_data[(wb_data['WB_OUT_DATETIME'] >= previous_start - datetime.timedelta(minutes = 15)) & (wb_data['WB_OUT_DATETIME'] <= starting_time)]
df_2 = df_2[df_2['SMS_ENTRY'].isna()]
df_2 = df_2[((df_2['SEC_TARE_DATETIME'] >= previous_start - datetime.timedelta(minutes = 15)) & (df_2['SEC_TARE_DATETIME'] <= previous_start)) == False]
df_2 = df_2[((df_2['WB_OUT_DATETIME'] >= starting_time - datetime.timedelta(minutes = 15)) & (df_2['WB_OUT_DATETIME'] <= starting_time)) == False]

In [667]:
df_2.head()

,SERIAL_NO,INSERT_DATE,MSG_FLAG,TRIP_NO,TL_NO,CAST_NO,DATE_IN,TIME_IN,DATE_OUT,TIME_OUT,...,MSG_STATUS,UPDATE_DATE,TORPEDO_FLAG,SHIFT,SMS_ENTRY,SMS_EXIT,BF_Entry,BF_Exit,WB_IN_DATETIME,WB_OUT_DATETIME


In [668]:
hm_df_new = pd.concat([df_1,df_2])
hm_in_new = hm_df_new['NET_WT'].sum()

In [669]:
hm_buffer = (hm_in_new + HM_left_in_SMS_tons) - hm_consumed
hm_buffer = max(hm_buffer, 0)

In [670]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=STEELDNA;'
                      'Database=DynaMix;'
                      'UID=sa;'
                      'PWD=admin@123;'
                      'Trusted_Connection=no;')

cursor = conn.cursor()

In [671]:
cursor.execute("UPDATE DynaMix.dbo.fixed_user_inputs SET HM_left_in_SMS_tons = ?", hm_buffer)
conn.commit()

In [672]:
sif_frl3_torpedo_df = sif_frl3_torpedo_df.apply(merging_torpedo_data, axis = 1)

In [673]:
sif_frl3_torpedo_df = sif_frl3_torpedo_df[(sif_frl3_torpedo_df['SMS_ENTRY'].isna()) & (sif_frl3_torpedo_df['SEC_TARE_DATETIME'].isna())]

In [674]:
sif_frl3_torpedo_df = sif_frl3_torpedo_df[(sif_frl3_torpedo_df['SUPPLY_AREA']=="HSM") | (sif_frl3_torpedo_df['SUPPLY_AREA']=="BF2/HSM") | (sif_frl3_torpedo_df['SUPPLY_AREA']=="BF 2/HSM") |(sif_frl3_torpedo_df['SUPPLY_AREA'].isna()) ]
sif_frl3_torpedo_df = sif_frl3_torpedo_df[(sif_frl3_torpedo_df['TL_NO'] <=15) | (sif_frl3_torpedo_df['TL_NO'] >=30) ]
sif_frl3_torpedo_df[['SERIAL_NO','INSERT_DATE','TL_NO','BF_Exit','WB_OUT_DATETIME','NET_WT','SUPPLY_AREA']]

,SERIAL_NO,INSERT_DATE,TL_NO,BF_Exit,WB_OUT_DATETIME,NET_WT,SUPPLY_AREA
888,149132.0,2024-02-22 11:18:13.647,8.0,NaT,2024-02-22 09:32:38,166.0,None
6375,139844.0,2023-08-23 17:20:56.953,11.0,NaT,2023-08-23 17:17:38,195.0,None
6376,139835.0,2023-08-23 17:14:48.270,32.0,NaT,2023-08-23 17:10:29,197.0,None
6377,139841.0,2023-08-23 16:32:47.040,33.0,NaT,2023-08-23 16:27:33,181.0,None
12603,133405.0,2023-05-02 12:17:03.797,8.0,NaT,2023-05-02 11:07:45,174.0,None
12606,133395.0,2023-05-02 12:11:09.127,6.0,NaT,2023-05-02 11:37:01,198.0,None
13217,132724.0,2023-04-21 12:39:03.550,13.0,NaT,2023-04-21 07:59:03,169.0,None
30101,1251.0,2022-04-20 14:48:54.190,1.0,NaT,2022-04-20 14:43:40,21.0,None
30484,1393.0,2022-04-16 17:27:49.013,3.0,NaT,2022-04-16 17:24:44,314.0,None
35584,106886.0,2022-02-15 21:18:06.460,2.0,NaT,2022-02-15 20:58:34,147.0,HSM


In [675]:
sif_frl3_torpedo_df_weigh_1 = sif_frl3_torpedo_df[(sif_frl3_torpedo_df['BF_Exit'].isna() == False) & (sif_frl3_torpedo_df['SMS_ENTRY'].isna())]

In [676]:
wb_window = datetime.timedelta(minutes = 20)

In [677]:
sif_frl3_torpedo_df_weigh_2 = sif_frl3_torpedo_df[
    (sif_frl3_torpedo_df['WB_OUT_DATETIME']<= starting_time)
    & (sif_frl3_torpedo_df['WB_OUT_DATETIME']>= starting_time - wb_window)
    & (sif_frl3_torpedo_df['BF_Exit'].isna())
    & (sif_frl3_torpedo_df['SMS_ENTRY'].isna())]

In [678]:
sif_frl3_torpedo_df_weigh_2

,SERIAL_NO,INSERT_DATE,MSG_FLAG,TRIP_NO,TL_NO,CAST_NO,DATE_IN,TIME_IN,DATE_OUT,TIME_OUT,...,MSG_STATUS,UPDATE_DATE,TORPEDO_FLAG,SHIFT,SMS_ENTRY,SMS_EXIT,BF_Entry,BF_Exit,WB_IN_DATETIME,WB_OUT_DATETIME


In [679]:
sif_frl3_torpedo_df_weigh = pd.concat([sif_frl3_torpedo_df_weigh_1, sif_frl3_torpedo_df_weigh_2])

In [680]:
sif_frl3_torpedo_df_weigh['BF_OUT_DATETIME'] = pd.NaT

# Keeping columns of interest
torpedo_data = sif_frl3_torpedo_df_weigh[['TL_NO','BF_OUT_DATETIME','WB_OUT_DATETIME','NET_WT']]

# Getting the torpedo data
torpedo_data.columns = ['Torpedo', 'BF Exit Time', 'Weight Bridge Arrival Time', 'Hot Metal']

# Adding Torpedo infront of the TLC number
# just data cleaning
torpedo_data['Torpedo'] = "Torpedo " + torpedo_data['Torpedo'].astype(str)

In [681]:
torpedo_data['Arrival Time'] = torpedo_data['Weight Bridge Arrival Time'].apply(lambda x: x + datetime.timedelta(minutes = 15) if bf_rate > 390 else x + datetime.timedelta(minutes = 20))
print('2:',torpedo_data)

2: Empty DataFrame
Columns: [Torpedo, BF Exit Time, Weight Bridge Arrival Time, Hot Metal, Arrival Time]
Index: []


In [682]:
print('3 SMS GEO FENCE:',sms_geofence)
print('4 BF GEO FENCE:',bf_geofence)

3 SMS GEO FENCE:           Torpedo          SMS Entry Time entry_status
30   Torpedo 14.0 2024-06-21 14:25:05.203          101
208   Torpedo 3.0 2024-06-21 13:45:05.040          103
4 BF GEO FENCE:           Torpedo            BF Exit Time Weight Bridge Arrival Time   
8    Torpedo 34.0 2024-06-21 14:30:05.257                             \
106  Torpedo 11.0 2024-06-21 14:05:05.120                              

    Hot Metal Arrival Time  
8                           
106                         


In [683]:
torpedo_data = torpedo_data[torpedo_data['Torpedo'].isin(sms_geofence['Torpedo'].unique()) == False]

# dropping torpedos from BF table if they have already arrived at WB
# so these torpedos have left BF and not arrived at WB
bf_geofence = bf_geofence[bf_geofence['Torpedo'].isin(torpedo_data['Torpedo'].unique()) == False]

In [685]:
# adding torpedos which are currently at BF
#torpedo_data = torpedo_data.append(bf_geofence,ignore_index=True)